In [1]:
import re
import string
import numpy as np
import pandas as pd

from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Embedding, GRU, Dense
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv("turkish_movie_sentiment_dataset.csv")
df.head()

,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\r\nbu adam...,Sevginin Gücü,"5,0"


In [3]:
df.shape

(83227, 3)

In [4]:
comments = lambda x : x[23:-24]

df["comment"] = df["comment"].apply(comments)
df["comment"].head()

0    Jean Reno denince zaten leon filmi gelir akla ...
1    Ekşın falan izlemek istiyorsanız eğer bunu izl...
2    Bu yapım hakkında öyle çok şey yazabilirim ki ...
3    finali yeter... (sting - shape of my heart)\r\...
4    Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...
Name: comment, dtype: object

In [5]:
floatize = lambda x : float(x[0:-2])

df["point"] = df["point"].apply(floatize)
df["point"].value_counts()

4.0    27463
5.0    15873
3.0    14494
2.0    13866
1.0     6381
0.0     5150
Name: point, dtype: int64

In [6]:
df.drop(df[df["point"] == 3].index, inplace = True)
df["point"] = df["point"].replace(1, 0)
df["point"] = df["point"].replace(2, 0)
df["point"] = df["point"].replace(4, 1)
df["point"] = df["point"].replace(5, 1)
df["point"].value_counts()

1.0    43336
0.0    25397
Name: point, dtype: int64

In [7]:
df.reset_index(inplace = True)
df.drop("index", axis = 1, inplace = True)
df.head()

,comment,film_name,point
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,1.0
4,Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,1.0


In [8]:
df["comment"] = df["comment"].apply(lambda x: x.lower())
df.head()

,comment,film_name,point
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,1.0
4,jean reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,1.0


In [9]:
def remove_punctuation(text):
    no_punc = [words for words in text if words not in string.punctuation]
    word_wo_punc = "".join(no_punc)
    return word_wo_punc

df["comment"] = df["comment"].apply(lambda x: remove_punctuation(x))
df["comment"] = df["comment"].apply(lambda x: x.replace("\r", " "))
df["comment"] = df["comment"].apply(lambda x: x.replace("\n", " "))

df.head()

,comment,film_name,point
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter sting shape of my heart bazıl...,Sevginin Gücü,1.0
4,jean reno bu adam kusursuz biri ve oyunculug...,Sevginin Gücü,1.0


In [10]:
def remove_numeric(corpus):
    output = "".join(words for words in corpus if not words.isdigit())
    return output

df["comment"] = df["comment"].apply(lambda x: remove_numeric(x)) 
df.head()

,comment,film_name,point
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter sting shape of my heart bazıl...,Sevginin Gücü,1.0
4,jean reno bu adam kusursuz biri ve oyunculug...,Sevginin Gücü,1.0


In [11]:
target = df["point"].values.tolist()
data = df["comment"].values.tolist()

cutoff = int(len(data)*0.80)

X_train, X_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [12]:
num_words = 10000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)
# tokenizer.word_index

In [13]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

print([X_train[1000]])
print(X_train_tokens[1000])

['film gerçekten bomba   çok komikti eğlenceli vakit geçirmek isteyen herkese öneririm  eddie murphy de gerçekten super oynamı']
[3, 35, 3225, 5, 1857, 128, 202, 672, 872, 269, 1641, 3232, 3495, 7, 35, 1327, 8852]


In [14]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([14, 47, 96, ..., 21,  4, 51])

In [15]:
np.mean(num_tokens)

31.318813379308338

In [16]:
np.max(num_tokens)

3094

In [17]:
max_tokens = np.mean(num_tokens) + (2*np.std(num_tokens))
max_tokens = int(max_tokens)
max_tokens

141

In [18]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9741754324705746

In [19]:
X_train_pad = pad_sequences(X_train_tokens, maxlen = max_tokens) 
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_tokens)

print(X_train_pad.shape)
print(X_test_pad.shape)

(54986, 141)
(13747, 141)


In [20]:
np.array(X_train_tokens[800])

array([  41,   36, 1864,  416,   54, 1824,  480,  355,   80,   33])

In [21]:
X_train_pad[2000]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    5,   70,    1,    3,    4,
       5061, 4071,  851,    5,    5,   16,  527, 2370, 1105])

In [22]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = " ".join(words) # Kelimeler aralarında boşluk bırakılarak ard arda yazılacaktır.
    return text

In [23]:
tokens_to_string(X_train_tokens[350])

'bence gayet güzel bi filmdi her ne kadar kendi dışına yinede bence mükemmel bi oyunculuk ama her seyrettiğimde ayrı güzel bu filmide korku kategorisine hayret ettim doğrusu ne'

In [24]:
embedding_size = 50
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = embedding_size, input_length = max_tokens, name = "embedding_layer"))
model.add(GRU(units = 16, return_sequences = True))
model.add(GRU(units = 8, return_sequences = True))
model.add(GRU(units = 4))
model.add(Dense(1, activation = "sigmoid"))

In [25]:
optimizer = Adam(lr = 1e-3)
model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 141, 50)           500000    
_________________________________________________________________
gru (GRU)                    (None, 141, 16)           3216      
_________________________________________________________________
gru_1 (GRU)                  (None, 141, 8)            600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 503,977
Trainable params: 503,977
Non-trainable params: 0
_________________________________________________________________


In [26]:
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

model.fit(X_train_pad, y_train, epochs = 5, batch_size = 256)

Epoch 1/5
215/215 [==============================] - 46s 213ms/step - loss: 0.5216 - accuracy: 0.7567
Epoch 2/5
215/215 [==============================] - 45s 208ms/step - loss: 0.4076 - accuracy: 0.8276
Epoch 3/5
215/215 [==============================] - 44s 207ms/step - loss: 0.3704 - accuracy: 0.8509
Epoch 4/5
215/215 [==============================] - 44s 204ms/step - loss: 0.3526 - accuracy: 0.8625
Epoch 5/5
215/215 [==============================] - 44s 205ms/step - loss: 0.3385 - accuracy: 0.8715


In [27]:
y_pred = model.predict(X_test_pad[0:1000])
y_pred = y_pred.T[0]
# y_pred

In [28]:
cls_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])
cls_true = np.array(y_test[0:1000])

In [29]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]
# incorrect

In [30]:
len(incorrect)

277

In [31]:
idx = incorrect[10]
X_test[idx]

'robert ludlumun romanları gerçekten sürükleyiciydibourne serisi sinamada bunu kanıtladıyazarın i̇kinci dünya savaşı ve özelliklede nazileri konu alan kitaplarıda çok güzeldirbu filmi henüz izleme fırsatımız olmadı ama tom cruisedan nasıl bir rus ajanı olur o tartışılı'

In [32]:
y_pred[idx]

0.87903625

In [33]:
cls_true[idx]

0.0